In [4]:
import yfinance as yf
import pandas as pd

In [5]:
tickers = [
    "AMZN", "GOOG", "TSLA", "CRM", "ADBE", "BIIB", "ISRG", "VRTX", "REGN",
    "ILMN", "EBAY", "BIDU", "NFLX", "BKNG", "LULU", "ADSK", "BRK-B", "BMRN",
    "ALGN", "META", "EA"
]

start_date = "2009-01-01"
end_date = "2015-11-30"

data = yf.download(
    tickers,
    start=start_date,
    end=end_date
)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  21 of 21 completed


In [8]:
def get_stock_data(data: pd.DataFrame, stock: str, freq: str = "D") -> pd.DataFrame:
    """
    Extracts data for a specific stock from a multi-level DataFrame and calculates percentage returns.

    Parameters:
    - data (pd.DataFrame): The multi-index stock data.
    - stock (str): The stock ticker to filter.
    - freq (str): Resampling frequency. Options: "D" (daily, default), "W" (weekly), "M" (monthly).

    Returns:
    - pd.DataFrame: A DataFrame containing data for the selected stock with a single-level index and percentage returns.
    """
    if stock not in data.columns.levels[1]:
        raise ValueError(f"Stock {stock} not found in the dataset.")
    
    # Extract single-stock data and flatten MultiIndex columns
    stock_data = data.xs(stock, axis=1, level=1).copy()
    
    # Calculate daily percentage returns
    stock_data["Pct_Return"] = stock_data["Close"].pct_change()

    # Resample if needed
    if freq == "W":
        stock_data = stock_data.resample("W").mean()  # Weekly averaged returns
    elif freq == "M":
        stock_data = stock_data.resample("M").mean()  # Monthly averaged returns

    return stock_data

In [9]:
get_stock_data(data, "AMZN", "D")

Price,Close,High,Low,Open,Volume,Pct_Return
Date,,,,,,
2009-01-02,2.718000,2.726500,2.553500,2.567500,145928000,NaN
2009-01-05,2.703000,2.787000,2.651500,2.786500,190196000,-0.005519
2009-01-06,2.868000,2.911000,2.687500,2.727500,221602000,0.061043
2009-01-07,2.810000,2.847500,2.767500,2.814500,158854000,-0.020223
2009-01-08,2.858000,2.866000,2.729000,2.749500,131558000,0.017082
...,...,...,...,...,...,...
2015-11-20,33.422501,33.443501,32.878502,33.396999,77922000,0.010858
2015-11-23,33.949501,34.138500,33.506001,33.575001,87702000,0.015768
2015-11-24,33.557499,33.790001,33.060501,33.707001,90868000,-0.011547
